<a href="https://colab.research.google.com/github/douniagh/MyApplication-/blob/main/DeepLearning1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import os
import cv2
import math
import random
import matplotlib.pyplot as plt
import shutil
from sklearn.preprocessing import QuantileTransformer
from PIL import Image
import warnings
warnings.filterwarnings("ignore")
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
import os
warnings.filterwarnings("ignore")
import keras
from keras import backend as K
from keras.models import Sequential
from keras.layers import Input, Dense, Conv1D, Dropout, MaxPool1D, Flatten
from sklearn.model_selection import train_test_split

In [3]:

# cnn model
from numpy import mean
from numpy import std
from numpy import dstack
from pandas import read_csv
from matplotlib import pyplot
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.utils import to_categorical

In [7]:
import pandas as pd

# Replace 'your_train_data.csv' and 'your_test_data.csv' with the filenames of your train and test data files
# Replace the appropriate paths if the files are in a different directory
train_file = '/content/drive/MyDrive/DataSplits/train_data.csv'
test_file = '/content/drive/MyDrive/DataSplits/test_data.csv'

# Load train data from CSV
train_df = pd.read_csv(train_file)

# Extract features and labels from train data
X_train = train_df.drop(columns=['Attack_type']).values
y_train = train_df['Attack_type'].values

# Load test data from CSV
test_df = pd.read_csv(test_file)

# Extract features and labels from test data
X_test = test_df.drop(columns=['Attack_type']).values
y_test = test_df['Attack_type'].values

# Print the shapes of the train and test data to verify
print("X_train shape:", X_train.shape)
print("y_train shape:", y_train.shape)
print("X_test shape:", X_test.shape)
print("y_test shape:", y_test.shape)


X_train shape: (15834598, 40)
y_train shape: (15834598,)
X_test shape: (3958650, 40)
y_test shape: (3958650,)


In [24]:
import numpy as np

# Assuming n_timesteps and n_features are pre-defined based on your dataset
n_timesteps = 1
n_features = 40

# Reshape X_train and X_test
X_train_reshaped = np.reshape(X_train, (X_train.shape[0], n_timesteps, n_features))
X_test_reshaped = np.reshape(X_test, (X_test.shape[0], n_timesteps, n_features))

# Print the new shapes of X_train and X_test to verify
print("X_train shape:", X_train_reshaped.shape)
print("X_test shape:", X_test_reshaped.shape)

X_train shape: (15834598, 1, 40)
X_test shape: (3958650, 1, 40)


In [ ]:
from keras.models import Sequential
from keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout

# Assuming you have already reshaped your data as mentioned before
# X_train shape: (19793248, 40, 1)
# X_test shape: (3958650, 40, 1)

# Build the 1D CNN model
model = Sequential()

# Add layers to the model
model.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(15834598,15834598, 40)))
model.add(Conv1D(filters=64, kernel_size=3, activation='relu'))
model.add(Dropout(0.5))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(100, activation='relu'))
model.add(Dense(15, activation='softmax'))

# Print the summary of the model
model.summary()


In [13]:
# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [19]:
from keras.utils import to_categorical
# Assuming 'y_train' and 'y_test' are the original string labels
label_mapping = {
    'Normal': 0,
    'DDoS_UDP': 1,
    'DDoS_ICMP': 2,
    'DDoS_TCP': 3,
    'Password': 4,
    'DDoS_HTTP': 5,
    'Vulnerability_scanner': 6,
    'SQL_injection': 7 ,
    'Backdoor': 8 ,
    'Uploading':  9,
    'Port_Scanning': 10,
    'XSS': 11  ,
    'Ransomware': 12,
    'MITM' :  13 ,
    'OS_Fingerprinting':  14
    # Add other classes and their corresponding integer values here
}

# Convert string labels to integer labels
y_train = [label_mapping[label] for label in y_train]
y_test = [label_mapping[label] for label in y_test]



In [ ]:

# Train the model
history = model.fit(X_train, y_train, batch_size=128, epochs=10, validation_split=0.1)


In [ ]:
# Evaluate the model on the test data
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f"Test Loss: {test_loss}, Test Accuracy: {test_accuracy}")

In [ ]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, Dropout, MaxPooling1D, Flatten, Dense

# Define the batch size
batch_size = 64

# Define a generator function for data batching
def data_generator(X, y, batch_size):
    num_samples = X.shape[0]
    indices = np.arange(num_samples)
    np.random.shuffle(indices)
    while True:
        for start_idx in range(0, num_samples, batch_size):
            end_idx = min(start_idx + batch_size, num_samples)
            excerpt = indices[start_idx:end_idx]
            yield X[excerpt], y[excerpt]

# Create the TCN model
model = Sequential()
model.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(19793248, 41)))
model.add(Conv1D(filters=64, kernel_size=3, activation='relu'))
model.add(Dropout(0.5))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(100, activation='relu'))
model.add(Dense(15, activation='softmax'))

# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Create generator instances
train_generator = data_generator(X_train, y_train, batch_size)
test_generator = data_generator(X_test, y_test, batch_size)

# Train the model using the generator
steps_per_epoch = X_train.shape[0] // batch_size
validation_steps = X_test.shape[0] // batch_size

model.fit(train_generator, epochs=epochs, steps_per_epoch=steps_per_epoch,
          validation_data=test_generator, validation_steps=validation_steps)


In [ ]:
df.Attack_type.value_counts()

Normal                   10066373
DDoS_UDP                  3201626
DDoS_ICMP                 2914354
DDoS_TCP                  2020120
Password                  1053385
DDoS_HTTP                  229022
Vulnerability_scanner      145869
SQL_injection               51203
Backdoor                    49724
Uploading                   37634
Port_Scanning               22564
XSS                         15915
Ransomware                  10925
MITM                         2458
OS_Fingerprinting            2002
Name: Attack_type, dtype: int64

## **Data Transformation**

Convert tabular data to images Procedures:

Use quantile transform to transform the original data samples

1  Élément de liste

into the scale of [0,255], representing pixel values
Generate images for each category (Normal, DoS, Fuzzy, Gear, RPM), each image consists of 27 data samples with 9 features. Thus, the size of each image is 993, length 9, width 9, and 3 color channels (RGB).

In [ ]:
!pip install dask dask-ml

In [ ]:
!pip install numpy==1.24.3

In [ ]:

import pandas as pd

# Assuming you have your DataFrame 'df'

# List of columns to drop
columns_to_drop = ['mqtt.conack.flags', 'arp.opcode', 'udp.port', 'udp.time_delta',
                   'dns.qry.name.len', 'dns.qry.qu', 'dns.qry.type', 'dns.retransmission',
                   'dns.retransmit_request', 'dns.retransmit_request_in', 'icmp.unused',
                   'mqtt.conflag.cleansess', 'mqtt.conflags', 'icmp.transmit_timestamp',
                   'mqtt.msg_decoded_as', 'mqtt.msg', 'mqtt.proto_len', 'mqtt.protoname',
                   'mqtt.topic', 'mqtt.topic_len', 'mbtcp.trans_id', 'mbtcp.unit_id']

# Drop the specified columns
df = df.drop(columns=columns_to_drop)


In [ ]:
from sklearn.preprocessing import MinMaxScaler

# Initialize the MinMaxScaler
scaler = MinMaxScaler()

# Define a function to apply scaling to a partition
def scale_partition(partition):
    for col in partition.columns:
        # Skip the label column
        if col == 'Attack_type':
            continue
        # Apply scaling to non-numeric columns
        if partition[col].dtype == 'int64':  # Assuming the encoded data type is int64
            partition[col] = scaler.fit_transform(partition[col].values.reshape(-1, 1))
    return partition

# Apply scaling to each partition in the DataFrame
scaled_df = df.map_partitions(scale_partition)

# Compute the scaled DataFrame
scaled_df = scaled_df.compute()

# Print the scaled DataFrame
print(scaled_df.head())

In [ ]:

from sklearn.preprocessing import MinMaxScaler,QuantileTransformer

# Initialize the MinMaxScaler
scaler = QuantileTransformer()

# Define a function to apply scaling to a partition
def scale_partition(partition):
    for col in partition.columns:
        # Skip the label column
        if col == 'Attack_type':
            continue
        # Apply scaling to non-numeric columns
        if partition[col].dtype == 'int64':  # Assuming the encoded data type is int64
            partition[col] = scaler.fit_transform(partition[col].values.reshape(-1, 1))
    return partition

# Apply scaling to each partition in the DataFrame
scaled_df = df.map_partitions(scale_partition)

# Compute the scaled DataFrame
scaled_df = scaled_df.compute()

# Print the scaled DataFrame
print(scaled_df.head())

In [ ]:
df.describe()

In [ ]:
import pandas as pd

# Define the file path
file_path = '/content/drive/MyDrive/dl_final.csv'

df = pd.read_csv(file_path,low_memory= False)
df.head()

In [ ]:
df.shape

(19793248, 41)

In [ ]:
import os
import pandas as pd
from sklearn.model_selection import train_test_split

# Replace 'your_data.csv' with the filename of your dataset
# Assuming you already have your data in a DataFrame named 'df'
# If you have not loaded your data into 'df', replace this with the appropriate code

# Extract features and labels
X = df.drop(columns=['Attack_type']).values
y = df['Attack_type'].values

# Output directory path
output_dir = '/content/drive/MyDrive/DataSplits/'

# Create the output directory if it doesn't exist
os.makedirs(output_dir, exist_ok=True)

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Output directory path
output_dir = '/content/drive/MyDrive/DataSplits/'

In [ ]:
# Function to save data to CSV file
def save_to_csv(X, y, filename):
    data = pd.DataFrame(data=np.column_stack((X, y)), columns=df.columns)
    data.to_csv(filename, index=False)

In [ ]:
# Save train data to CSV incrementally
train_file = os.path.join(output_dir, 'train_data.csv')

# Chunk size for incremental saving
chunk_size = 100000

# Save data in chunks
num_samples = X_train.shape[0]
for i in range(0, num_samples, chunk_size):
    start_idx = i
    end_idx = min(i + chunk_size, num_samples)

    train_chunk = pd.DataFrame(data=np.column_stack((X_train[start_idx:end_idx], y_train[start_idx:end_idx])), columns=df.columns)

    if i == 0:
        # For the first chunk, include the header (column names)
        train_chunk.to_csv(train_file, index=False, mode='a', header=True)
    else:
        # For subsequent chunks, exclude the header to avoid writing duplicate column names
        train_chunk.to_csv(train_file, index=False, mode='a', header=False)

    print(f"Chunk {i//chunk_size + 1} saved.")

print("Train data saved incrementally.")

In [ ]:
# Save test data to CSV incrementally
test_file = os.path.join(output_dir, 'test_data.csv')

# Chunk size for incremental saving
chunk_size = 100000

# Save data in chunks
num_samples = X_test.shape[0]
for i in range(0, num_samples, chunk_size):
    start_idx = i
    end_idx = min(i + chunk_size, num_samples)

    test_chunk = pd.DataFrame(data=np.column_stack((X_test[start_idx:end_idx], y_test[start_idx:end_idx])), columns=df.columns)

    if i == 0:
        # For the first chunk, include the header (column names)
        test_chunk.to_csv(test_file, index=False, mode='a', header=True)
    else:
        # For subsequent chunks, exclude the header to avoid writing duplicate column names
        test_chunk.to_csv(test_file, index=False, mode='a', header=False)

    print(f"Chunk {i//chunk_size + 1} saved.")

print("Test data saved incrementally.")

In [ ]:
# Just printing out the dimentions to verify the data
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(15834598, 40) (15834598,)
(3958650, 40) (3958650,)


In [ ]:
# Build the model

# The model architecture type is sequential hence that is used
model = Sequential()

# We are using 4 convolution layers for feature extraction
model.add(Conv1D(filters=512, kernel_size=32, padding='same', kernel_initializer='normal', activation='relu', input_shape=(19793248,41)))
model.add(Conv1D(filters=512, kernel_size=32, padding='same', kernel_initializer='normal', activation='relu'))
model.add(Dropout(0.2)) # This is the dropout layer. It's main function is to inactivate 20% of neurons in order to prevent overfitting
model.add(Conv1D(filters=256, kernel_size=32, padding='same', kernel_initializer='normal', activation='relu'))
model.add(Dropout(0.2))
model.add(Conv1D(filters=256, kernel_size=32, padding='same', kernel_initializer='normal', activation='relu'))
model.add(MaxPool1D(pool_size=128)) # We use MaxPooling with a filter size of 128. This also contributes to generalization
model.add(Dropout(0.2))

# The prevous step gices an output of multi dimentional data, which cannot be fead directly into the feed forward neural network. Hence, the model is flattened
model.add(Flatten())
# One hidden layer of 128 neurons have been used in order to have better classification results
model.add(Dense(units=128, kernel_initializer='normal', activation='relu'))
model.add(Dropout(0.5))
# The final neuron HAS to be 1 in number and cannot be more than that. This is because this is a binary classification problem and only 1 neuron is enough to denote the class '1' or '0'
model.add(Dense(units=15, activation='softmax'))

# Print the summary of the model
model.summary()

In [ ]:
df.shape

(19793248, 41)

In [ ]:
model = Sequential()
model.add(Conv1D(filters=64, kernel_size=41, activation='relu', input_shape=(19793248,41)))
model.add(Conv1D(filters=64, kernel_size=41, activation='relu'))
model.add(Dropout(0.5))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(100, activation='relu'))
model.add(Dense(15, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

NameError: ignored

In [ ]:
# Just printing out the dimentions to verify the data
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

## **Generate images for each class**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
pip install torch==1.9.1

ERROR: Could not find a version that satisfies the requirement torch==1.9.1 (from versions: 1.11.0, 1.12.0, 1.12.1, 1.13.0, 1.13.1, 2.0.0, 2.0.1)
ERROR: No matching distribution found for torch==1.9.1


In [ ]:
pip -q install git+https://github.com/alok-ai-lab/pyDeepInsight.git#egg=pyDeepInsight

  Preparing metadata (setup.py) ... done


In [ ]:
! pip install tensorflow
! pip install keras
! pip install numpy


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
import os
warnings.filterwarnings("ignore")
import keras
from keras import backend as K
from keras.models import Sequential
from keras.layers import Input, Dense, Conv1D, Dropout, MaxPool1D, Flatten
from sklearn.model_selection import train_test_split

In [ ]:
# This is not a mandatory step
# This is used to set the figure size so that the plots appear to be of that size
# If this parameter is not set, then the default plot size is used
plt.rcParams['figure.figsize'] = 30, 15

In [ ]:
# This step is actually supposed to be used to randomly split dataset into training and testing set as well as to shuffle them
# However, here I am using it just to shuffle the dataset and therefore the test_size is equated to 0
X_train, _, y_train, _ = train_test_split(X_train, y_train, test_size=0, random_state=0)